In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

In [11]:
#Loading
df = pd.read_csv("job_postings.csv")
print(df.columns)
display(df.head())

Index(['Job Posting ID', 'Job Posting Date', 'Job Title', 'Job Title Full',
       'Job Title Additional Info', 'Job Position Type', 'Job Position Level',
       'Years of Experience', 'Job Skills', 'Job Location', 'Minimum Pay',
       'Maximum Pay', 'Pay Rate', 'Number of Applicants', 'Company Name',
       'Company Industry', 'Company Size'],
      dtype='object')


,Job Posting ID,Job Posting Date,Job Title,Job Title Full,Job Title Additional Info,Job Position Type,Job Position Level,Years of Experience,Job Skills,Job Location,Minimum Pay,Maximum Pay,Pay Rate,Number of Applicants,Company Name,Company Industry,Company Size
0,2701524240,2017-01-01,Software Engineer,Software Engineer,Java Full Stack | Remote,Full-time,Entry level,1,"database, javascript, agile, linux, server, no...",United States,NaN,NaN,NaN,6.0,"Cardinal Financial Company, Limited Partnership",Financial Services,"1,001-5,000 employees"
1,2719108338,2017-01-01,Data Engineer,Senior Azure Data Engineer,NaN,Full-time,Mid-Senior level,2,"data_lake, cloud, python, spark, github, wareh...",United States,NaN,NaN,NaN,1.0,Brinks Home,Consumer Electronics,"1,001-5,000 employees"
2,2719503370,2017-01-01,Software Engineer,Software Engineer I,NaN,Full-time,Entry level,5,"mongo, oracle, microsoft, css, javascript, htm...",United States,NaN,NaN,NaN,16.0,Paycor,Computer Software,"1,001-5,000 employees"
3,2734877741,2017-01-01,Business Analyst,Associate Business Analyst,Telecommute,Full-time,Entry level,2,"agile, excel","Phoenix, AZ",NaN,NaN,NaN,1.0,Optum,Hospital & Health Care,"10,001+ employees"
4,2752415616,2017-01-01,Developer,Swift Developer,NaN,Contract,Mid-Senior level,4,"excel, back-end, ios, swift, programming","Richmond, CA",NaN,NaN,NaN,NaN,Toptal,Internet,"1,001-5,000 employees"


In [12]:
# Filtering for data-related jobs only
data_jobs = [
    'Data Scientist',
    'Data Analyst',
    'Data Engineer',
    'Business Analyst',
    'Business Intelligence Analyst',
    'Machine Learning Engineer',
    'Senior Analyst',
    'Financial Analyst'
]

df = df[df['Job Title'].isin(data_jobs)].copy()

In [13]:
#Converting date to datetime format
df['Job Posting Date'] = pd.to_datetime(df['Job Posting Date'])

# Adding time columns
df['Year'] = df['Job Posting Date'].dt.year
df['Month'] = df['Job Posting Date'].dt.month
df['Year-Month'] = df['Job Posting Date'].dt.strftime('%Y-%m')

In [14]:
#Cleaning location
def clean_location(loc):
    if pd.isna(loc):
        return 'Unknown'
    loc = str(loc).strip()
    if loc == 'United States':
        return 'United States (Remote)'
    if 'Greater' in loc:
        loc = loc.replace('Greater ', '')
    return loc

df['Job Location Clean'] = df['Job Location'].apply(clean_location)

# Extract state
def extract_state(loc):
    if pd.isna(loc) or 'United States' in str(loc):
        return 'Remote'
    if ',' in str(loc):
        parts = str(loc).split(',')
        state = parts[-1].strip().split()[0]
        if len(state) == 2:
            return state
    return 'Other'

df['State'] = df['Job Location'].apply(extract_state)

In [15]:
# Cleaning salary
df['Average Salary'] = (df['Minimum Pay'] + df['Maximum Pay']) / 2
df['Has Salary Info'] = df['Minimum Pay'].notna()

In [16]:
# Cleaning skills
df['Job Skills'] = df['Job Skills'].fillna('Not Specified')
df['Number of Skills'] = df['Job Skills'].apply(
    lambda x: len(str(x).split(',')) if x != 'Not Specified' else 0
)

In [17]:
# Cleaning company
df['Company Name'] = df['Company Name'].fillna('Unknown')
df['Company Industry'] = df['Company Industry'].fillna('Unknown')
df['Company Size'] = df['Company Size'].fillna('Unknown')

In [18]:
# Experience categories
def categorize_experience(years):
    if pd.isna(years):
        return 'Not Specified'
    years = int(years)
    if years == 0:
        return 'Entry Level'
    elif years <= 2:
        return '1-2 years'
    elif years <= 5:
        return '3-5 years'
    elif years <= 10:
        return '6-10 years'
    else:
        return '10+ years'

df['Experience Category'] = df['Years of Experience'].apply(categorize_experience)

In [19]:
# 5. Saving csv
df.to_csv('job_postings_CLEAN.csv', index=False)
